<a href="https://colab.research.google.com/github/BlackMagicAI/Cloud-Vision-Api/blob/main/notebooks/CloudVisionApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goolge Cloud Vision Api
ref:
[Docs](https://cloud.google.com/vision/docs/object-localizer)

In [ ]:
# Make image input and output directories
!mkdir images
!mkdir out
# Load default test image
!wget https://raw.githubusercontent.com/BlackMagicAI/Tiny-Yolo-3/master/example/images/dog.jpg -P images

###Use python request library

**Set Google Application Credentials environment**

In [1]:
import os
import subprocess
# set system environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/aiml9-xor-327423-ed14aa16f610.json'
# get bearer token from command line gcloud 
bearer_token=subprocess.run(["gcloud", "auth", "application-default", "print-access-token"],stdout=subprocess.PIPE, text=True)

In [5]:
# Print bearer token value to file output
with open('token.txt', 'w') as writefile:
    writefile.write(bearer_token.stdout.strip())

In [ ]:
import base64
import json
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
# Using Python Request library
import requests

# Pass the image data to an encoding function.
def encode_image(image):
  infile = BytesIO()
  image.save(infile, format="JPEG")
  return base64.b64encode(infile.getvalue()).decode('utf-8')

# Draw bounding box, class labels and scores onto original image
def drawBoundingBox(image, boxes, all_classes, scores, color):
  # fnt =ImageFont.truetype('./fonts/Ubuntu-Bold.ttf', 18)
  fnt = ImageFont.load_default()
  draw = ImageDraw.Draw(image)
  for box, cl, score in zip(boxes, classes, scores):
    p0, p1, p2, p3 = box
    x0, y0 = p0
    x1, y1 = p2

    top = x0 * image.width # max(0, np.floor(x + 0.5).astype(int))
    left = y0 * image.height # max(0, np.floor(y + 0.5).astype(int))
    right = x1 * image.width # min(image.size[0], np.floor(x + w + 0.5).astype(int))
    bottom = y1 * image.height # min(image.size[1], np.floor(y + h + 0.5).astype(int))

    draw.rectangle([(top, left), (right, bottom)], outline=color)#blue rectangle
    draw.text((top + 4, left), '{0} {1:.2f}'.format(cl, score), font=fnt, fill=(0, 0, 255))

# Process json response
def processJson(response):
  boxes = []
  classes = []
  scores = []
  for responseObject in response['responses']:    
    for localizedObjectAnnotation in responseObject['localizedObjectAnnotations']:
      classes.append(localizedObjectAnnotation['name'])
      scores.append(localizedObjectAnnotation['score'])
      box = []
      for normalizedVertice in localizedObjectAnnotation['boundingPoly']['normalizedVertices']:
        box.append((normalizedVertice['x'], normalizedVertice['y']))
      boxes.append(box) 
  return boxes, classes, scores    

**Form JSON body and send POST request**

In [ ]:
# Load image
input_image_name = 'dog.jpg'
imagedata = Image.open("images/" + input_image_name)
print(input_image_name)

# Save json data as local file
jsonBody = {"requests": [
    {
      "image": {
        "content": encode_image(imagedata)
      },
      "features": [
        {
          "maxResults": 10,
          "type": "OBJECT_LOCALIZATION"
        }
      ]
    }
  ]
}

# define variables
BEARER_TOKEN = bearer_token.stdout.strip()
url = 'https://vision.googleapis.com/v1/images:annotate'
bearer = 'Bearer {}'.format(BEARER_TOKEN)

# send POST request to url
response = requests.post(url, 
                  headers={'Content-Type': 'application/json; charset=utf-8', 
                           'Authorization': bearer},
                  json=jsonBody
                  )
# print(response.json()['responses'])

### Process results and display annotated image

In [ ]:
# Draw bounding boxes on original image
boxes, classes, scores = processJson(response.json())
drawBoundingBox(imagedata, boxes, classes, scores, (0,255,0))
display(imagedata)

###Or Use the Curl command

In [ ]:
%%bash
export GOOGLE_APPLICATION_CREDENTIALS="<INSERT-CREDENTIALS-FILE-PATH-HERE>"

# load image and base64 encode it
jsonbody=$(base64 images/dog.jpg)

# form json body
INPUT_DATA_FILE='{"requests": [
    {
      "image": {
        "content": "jsonbody_placeholder"
      },
      "features": [
        {
          "maxResults": 10,
          "type": "OBJECT_LOCALIZATION"
        }
      ]
    }
  ]
}'

# substitute base64 encoded image string into json body
INPUT_DATA_FILE=${INPUT_DATA_FILE/jsonbody_placeholder/$jsonbody}
# write json body to a file
echo $INPUT_DATA_FILE > request.json

res=$(curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
"https://vision.googleapis.com/v1/images:annotate")
echo "$res"

###Or Use the Curl command from Python and process results

In [ ]:
results=subprocess.check_output(['curl', '-X', 'POST', '-H', 'Authorization: Bearer {}'.format(BEARER_TOKEN),'-H', 'Content-Type: application/json; charset=utf-8','-d',json.dumps(jsonBody) , 'https://vision.googleapis.com/v1/images:annotate']).decode('UTF-8')
# Draw bounding boxes on original image
resultsObj = json.loads(results) # convert results string response to Python object
boxes, classes, scores = processJson(resultsObj)
drawBoundingBox(imagedata, boxes, classes, scores, (0,255,0))
display(imagedata)